<a href="https://colab.research.google.com/github/dviva1972/denvlaiva/blob/master/dll_hw__5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLL

## Домашняя работа 5  |  RNN
## Иванов Денис

## Задание 1

1. Напишите алгоритм шифра цезаря для генерации выборки 

2.  Создайте и обучите нейронную сеть (вход - зашифрованная фраза, выход - дешифрованная фраза)

3. Проверьте качество модели

In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time

import random

1.Алгоритм шифра Цезаря 

In [2]:
def Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [3]:
Сaesar('trew', 2)

'vtgy'

Сгенерим данные на базе изречений симпсонов

In [4]:
import os
import re
import sys
from google.colab import drive 

In [5]:
import copy

In [6]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/My Drive/DLL/simpsons/data.csv', encoding='utf-8-sig').iloc[:,[-2]]
df = df.dropna(subset=['normalized_text'])

df['shift']   = [random.randint(1, 10) for i in range(len(df))]
df['text_in'] = [' '.join(re.findall('[\w]+', i)) for i in df['normalized_text']]
df['text_out']= df.loc[:, ['text_in', 'shift']].apply(
                        lambda row: Сaesar(row['text_in'], row['shift']), axis=1)

df = df.iloc[:,1:]
df.head()

,shift,text_in,text_out
0,9,maggie look whats that,vjpprn uxxt fqjcb cqjc
1,4,lee mur lee mur,pii qyv pii qyv
2,6,zee boo zee boo,fkk huu fkk huu
3,2,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,2,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...


In [9]:
[[c for c in ph] for ph in df.text_in if type(ph) is not str]

[]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train, test = train_test_split(df, test_size=0.2)
train_text  = train['text_out' ].tolist()
train_label = train['text_in'].tolist()
test_text   = test[ 'text_out' ].tolist()
test_label  = test[ 'text_in'].tolist()

In [12]:
INDEX_TO_CHAR = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

MAX_LEN       = 20

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

In [13]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [14]:
model = RNN_Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
epochs = 20

In [15]:
for epoch in range(epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100]
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten()
        
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_test.flatten())
        test_loss += loss.item()
        test_passed += 1

    print(f"Epoch {epoch + 1}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 1. Time: 2.948, Train loss: 1.687, Test loss: 1.284145
Epoch 2. Time: 2.604, Train loss: 1.254, Test loss: 1.181833
Epoch 3. Time: 2.621, Train loss: 1.162, Test loss: 1.090840
Epoch 4. Time: 2.578, Train loss: 1.034, Test loss: 0.934189
Epoch 5. Time: 2.684, Train loss: 0.856, Test loss: 0.778867
Epoch 6. Time: 2.605, Train loss: 0.716, Test loss: 0.700779
Epoch 7. Time: 2.629, Train loss: 0.596, Test loss: 0.603406
Epoch 8. Time: 2.577, Train loss: 0.514, Test loss: 0.567529
Epoch 9. Time: 2.573, Train loss: 0.454, Test loss: 0.494375
Epoch 10. Time: 2.626, Train loss: 0.408, Test loss: 0.473532
Epoch 11. Time: 2.587, Train loss: 0.386, Test loss: 0.419784
Epoch 12. Time: 2.589, Train loss: 0.349, Test loss: 0.417215
Epoch 13. Time: 2.603, Train loss: 0.301, Test loss: 0.367100
Epoch 14. Time: 2.609, Train loss: 0.267, Test loss: 0.344502
Epoch 15. Time: 2.533, Train loss: 0.245, Test loss: 0.334135
Epoch 16. Time: 2.618, Train loss: 0.234, Test loss: 0.328219
Epoch 17. Time: 2

###Задание 2.

На основе лекционного ноутбука.

а) построить RNN-ячейку на основе полносвязных слоев

б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [16]:
X = convert_to_torch(df['text_in'].tolist())

MAX_LEN = 50

In [17]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 812, batch_first=True)
        self.linear = torch.nn.Linear(812, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [18]:
def generate_sentence(txt_list):

    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(dev)
    
    for j,w in enumerate(sentence):
        if j>=MAX_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0).unsqueeze(1)], axis=1)
        ww = INDEX_TO_CHAR[w]
        if ww == 'none':
            break

        sentence.append(ww)

    return ''.join(sentence)

In [19]:
model     = Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.0001)

phrase = 'youre a great mom you were always '

In [20]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100] 
        X_batch = batch[:, :-1].to(dev)
        Y_batch = batch[:, 1:].flatten().to(dev) 

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch).to(dev)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1     

    if ep%5 == 0: 
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence(phrase)
        print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='') 


Epoch 0. Time: 1.396, Train loss: 2.577
youre a great mom you were always io the  ao the  ao the  ao the  ao the  ao the  ao
Epoch 4, loss: 1.930
Epoch 5. Time: 1.159, Train loss: 1.886
youre a great mom you were always i gont in the sont in the sont in the sont in the 
Epoch 9, loss: 1.755
Epoch 10. Time: 1.162, Train loss: 1.728
youre a great mom you were always are the sont the sont the sont the sont the sont t
Epoch 14, loss: 1.633
Epoch 15. Time: 1.151, Train loss: 1.613
youre a great mom you were always a mant the some the some the some the some the som
Epoch 19, loss: 1.539
Epoch 20. Time: 1.156, Train loss: 1.522
youre a great mom you were always dont want to the some on the bart the really sore 
Epoch 24, loss: 1.463
Epoch 25. Time: 1.153, Train loss: 1.450
youre a great mom you were always dad the sere stop the bart the really sore what ar
Epoch 29, loss: 1.401
Epoch 30. Time: 1.151, Train loss: 1.390
youre a great mom you were always the list sare i cant believe the bart i 